<a href="https://colab.research.google.com/github/sanuthit/Risk-Based-Motor-Insurance-Premium-Calculation-System-/blob/risk-model-development/accident_risk_DLmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, average_precision_score
)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root = "/content/drive/MyDrive"
DATA_DIR = "/content/drive/MyDrive/Data/Datasets"


df = pd.read_csv("/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_cleaned.csv", encoding="utf-8")

DATA_PATH = "/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_cleaned.csv"
df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(60000, 49)


,policy_id,customer_id,driver_age,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,ncb_percentage,accidents_last_3_years,...,had_claim_within_1_year,num_claims_within_1_year,total_claim_amount_within_1_year,hard_flag_blacklist,driver_age_band,vehicle_age_band,risk_exposure_proxy,doc_missing_score,compliance_risk_score,ncb_validity_flag
0,P000001,C00002,62,M,Businessman,4,0,1,35,0,...,0,0,0,0,60+,4–7,Low,0,0,0
1,P000002,C00003,69,F,Businessman,20,0,1,20,0,...,0,0,0,0,60+,13+,Low,0,0,0
2,P000003,C00004,37,M,IT Engineer,13,0,0,0,0,...,0,0,0,0,35–44,13+,Low,0,0,0
3,P000004,C00005,63,F,Businessman,31,0,1,20,1,...,0,0,0,0,60+,13+,High,1,0,0
4,P000005,C00006,21,M,Student,0,0,0,0,1,...,0,0,0,0,18–24,13+,Low,0,0,0


In [ ]:
RISK_FEATURES = [
    "driver_age", "driver_age_band", "driver_gender", "driver_occupation",
    "years_of_driving_experience", "member_automobile_assoc_ceylon",
    "has_previous_motor_policy", "accidents_last_3_years", "ncb_percentage",
    "vehicle_type", "vehicle_segment", "engine_capacity_cc", "fuel_type",
    "vehicle_age_years", "vehicle_age_band", "has_lpg_conversion",
    "vehicle_usage_type", "risk_exposure_proxy", "registration_district",
    "parking_type", "doc_missing_score", "compliance_risk_score"
]

TARGET = "had_claim_within_1_year"

In [ ]:
df_risk = df[RISK_FEATURES + [TARGET]].copy()

In [ ]:
X = df_risk[RISK_FEATURES]
y = df_risk[TARGET]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print("Data shapes:")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"Target distribution: {y_train.value_counts(normalize=True).to_dict()}")

Data shapes:
Train: (42000, 22), Val: (9000, 22), Test: (9000, 22)
Target distribution: {0: 0.7499285714285714, 1: 0.25007142857142856}


In [ ]:
categorical_cols = [
    "driver_age_band", "driver_gender", "driver_occupation",
    "vehicle_type", "vehicle_segment", "fuel_type",
    "vehicle_age_band", "vehicle_usage_type", "risk_exposure_proxy",
    "registration_district", "parking_type"
]

continuous_cols = [
    "driver_age", "years_of_driving_experience",
    "member_automobile_assoc_ceylon", "has_previous_motor_policy",
    "accidents_last_3_years", "ncb_percentage",
    "engine_capacity_cc", "vehicle_age_years", "has_lpg_conversion",
    "doc_missing_score", "compliance_risk_score"
]

EVALUATION

In [ ]:
def evaluate_model_comprehensive(y_true, proba, model_name="Model", threshold=None):
    """Comprehensive evaluation with optimal threshold finding"""

    # ROC-AUC and PR-AUC
    roc_auc = roc_auc_score(y_true, proba)
    pr_auc = average_precision_score(y_true, proba)

    # Find optimal threshold if not provided
    if threshold is None:
        thresholds = np.arange(0.05, 0.95, 0.01)
        f1_scores = []
        for t in thresholds:
            pred = (proba >= t).astype(int)
            f1_scores.append(f1_score(y_true, pred, zero_division=0))

        best_idx = np.argmax(f1_scores)
        threshold = thresholds[best_idx]
        best_f1 = f1_scores[best_idx]
    else:
        pred = (proba >= threshold).astype(int)
        best_f1 = f1_score(y_true, pred, zero_division=0)

    # Calculate metrics at best threshold
    pred = (proba >= threshold).astype(int)
    precision = precision_score(y_true, pred, zero_division=0)
    recall = recall_score(y_true, pred, zero_division=0)

    print(f"\n{'='*80}")
    print(f"{model_name} - EVALUATION RESULTS")
    print(f"{'='*80}")
    print(f"ROC-AUC:           {roc_auc:.4f}")
    print(f"PR-AUC:            {pr_auc:.4f}")
    print(f"Optimal Threshold: {threshold:.3f}")
    print(f"F1-Score:          {best_f1:.4f}")
    print(f"Precision:         {precision:.4f}")
    print(f"Recall:            {recall:.4f}")
    print(f"\nConfusion Matrix:")
    print(confusion_matrix(y_true, pred))
    print(f"\nClassification Report:")
    print(classification_report(y_true, pred, digits=4))

    return {
        'model': model_name,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'threshold': threshold,
        'f1': best_f1,
        'precision': precision,
        'recall': recall
    }

# 01. FT-Transformer

In [ ]:
#!pip install pytorch-tabular torchmetrics

In [ ]:
import torch
import torch.serialization
from omegaconf.dictconfig import DictConfig
torch.serialization.add_safe_globals([DictConfig])

from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.ft_transformer.config import FTTransformerConfig

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
val_df = pd.concat([X_val, y_val], axis=1).reset_index(drop=True)
test_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

# Set dtypes
train_df[continuous_cols] = train_df[continuous_cols].astype('float32')
val_df[continuous_cols] = val_df[continuous_cols].astype('float32')
test_df[continuous_cols] = test_df[continuous_cols].astype('float32')

train_df[categorical_cols] = train_df[categorical_cols].astype('category')
val_df[categorical_cols] = val_df[categorical_cols].astype('category')
test_df[categorical_cols] = test_df[categorical_cols].astype('category')

In [ ]:
data_config = DataConfig(
    target=[TARGET],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config = TrainerConfig(
    max_epochs=50,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    batch_size=1024,
    early_stopping="valid_loss",
    early_stopping_patience=10,
    load_best=False,
    checkpoints=None
)

optimizer_config = OptimizerConfig()

ft_model_config = FTTransformerConfig(
    task="classification",
    learning_rate=1e-3,
    num_heads=8,
    num_attn_blocks=4,
    attn_dropout=0.2,
    ff_dropout=0.2,
    embedding_dims=[(col, 16) for col in categorical_cols],
)

ft_transformer = TabularModel(
    data_config=data_config,
    model_config=ft_model_config,
    trainer_config=trainer_config,
    optimizer_config=optimizer_config,
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# Train FT-Transformer
print("\nTraining FT-Transformer...")
ft_transformer.fit(train=train_df, validation=val_df)

# Predict on validation
val_pred_ft = ft_transformer.predict(val_df)
prob_col = [c for c in val_pred_ft.columns if "probability" in c.lower()][0]
val_proba_ft = val_pred_ft[prob_col].values

# Evaluate on validation
results_ft_val = evaluate_model_comprehensive(
    y_val, val_proba_ft, model_name="FT-Transformer (Validation)"
)

Epoch 14/49 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42/42 0:01:40 • 0:00:00 0.44it/s v_num: 3.000 train_loss: 0.568     
                                                                                valid_loss: 0.538 valid_accuracy:  
                                                                                0.749 train_accuracy: 0.751        

INFO:pytorch_tabular.tabular_model:Training the model completed



FT-Transformer (Validation) - EVALUATION RESULTS
ROC-AUC:           0.3529
PR-AUC:            0.1872
Optimal Threshold: 0.050
F1-Score:          0.4001
Precision:         0.2501
Recall:            1.0000

Confusion Matrix:
[[   0 6749]
 [   0 2251]]

Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      6749
           1     0.2501    1.0000    0.4001      2251

    accuracy                         0.2501      9000
   macro avg     0.1251    0.5000    0.2001      9000
weighted avg     0.0626    0.2501    0.1001      9000



In [ ]:
# Predict on test
test_pred_ft = ft_transformer.predict(test_df)
test_proba_ft = test_pred_ft[prob_col].values

# Evaluate on test
results_ft_test = evaluate_model_comprehensive(
    y_test, test_proba_ft,
    threshold=results_ft_val['threshold'],
    model_name="FT-Transformer (Test)"
)


FT-Transformer (Test) - EVALUATION RESULTS
ROC-AUC:           0.3540
PR-AUC:            0.1863
Optimal Threshold: 0.050
F1-Score:          0.4000
Precision:         0.2500
Recall:            1.0000

Confusion Matrix:
[[   0 6750]
 [   0 2250]]

Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      6750
           1     0.2500    1.0000    0.4000      2250

    accuracy                         0.2500      9000
   macro avg     0.1250    0.5000    0.2000      9000
weighted avg     0.0625    0.2500    0.1000      9000



# 02. NGBOOST

In [ ]:
!pip install ngboost

In [77]:
from ngboost import NGBClassifier
from ngboost.distns import Bernoulli
from sklearn.preprocessing import LabelEncoder

In [78]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_train_encoded[categorical_cols] = oe.fit_transform(X_train[categorical_cols].astype(str))
X_val_encoded[categorical_cols]   = oe.transform(X_val[categorical_cols].astype(str))
X_test_encoded[categorical_cols]  = oe.transform(X_test[categorical_cols].astype(str))

In [79]:
print("\nTraining NGBoost...")
ngboost_model = NGBClassifier(
    Dist=Bernoulli,
    n_estimators=500,
    learning_rate=0.01,
    minibatch_frac=1.0,
    verbose=True,
    verbose_eval=100,
    random_state=42
)

ngboost_model.fit(
    X_train_encoded.values,
    y_train.values,
    X_val=X_val_encoded.values,
    Y_val=y_val.values,
    early_stopping_rounds=50
)

# Predict on validation
val_proba_ngb = ngboost_model.predict_proba(X_val_encoded.values)[:, 1]

# Evaluate on validation
results_ngb_val = evaluate_model_comprehensive(
    y_val, val_proba_ngb, model_name="NGBoost (Validation)"
)


Training NGBoost...
[iter 0] loss=0.5624 val_loss=0.5621 scale=1.0000 norm=2.0000
[iter 100] loss=0.5406 val_loss=0.5406 scale=1.0000 norm=1.9631
[iter 200] loss=0.5364 val_loss=0.5374 scale=1.0000 norm=1.9674
[iter 300] loss=0.5339 val_loss=0.5358 scale=1.0000 norm=1.9715
[iter 400] loss=0.5323 val_loss=0.5351 scale=1.0000 norm=1.9754

NGBoost (Validation) - EVALUATION RESULTS
ROC-AUC:           0.6517
PR-AUC:            0.3786
Optimal Threshold: 0.210
F1-Score:          0.4445
Precision:         0.3125
Recall:            0.7699

Confusion Matrix:
[[2936 3813]
 [ 518 1733]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8500    0.4350    0.5755      6749
           1     0.3125    0.7699    0.4445      2251

    accuracy                         0.5188      9000
   macro avg     0.5813    0.6025    0.5100      9000
weighted avg     0.7156    0.5188    0.5428      9000



In [80]:
test_proba_ngb = ngboost_model.predict_proba(X_test_encoded.values)[:, 1]

# Evaluate on test
results_ngb_test = evaluate_model_comprehensive(
    y_test, test_proba_ngb,
    threshold=results_ngb_val['threshold'],
    model_name="NGBoost (Test)"
)


NGBoost (Test) - EVALUATION RESULTS
ROC-AUC:           0.6464
PR-AUC:            0.3661
Optimal Threshold: 0.210
F1-Score:          0.4385
Precision:         0.3081
Recall:            0.7604

Confusion Matrix:
[[2908 3842]
 [ 539 1711]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8436    0.4308    0.5704      6750
           1     0.3081    0.7604    0.4385      2250

    accuracy                         0.5132      9000
   macro avg     0.5759    0.5956    0.5045      9000
weighted avg     0.7098    0.5132    0.5374      9000



In [ ]:
print("\n" + "="*80)
print("NGBOOST UNCERTAINTY (Classification)")
print("="*80)

X_sub = X_test_encoded.values[:100]
dist = ngboost_model.pred_dist(X_sub)

# Bernoulli probs → shape (2, N) → transpose
probs = dist.probs.T
p_claim = probs[:, 1]

# Approx 95% CI around predicted probability
z = 1.96
lower = np.clip(p_claim - z*np.sqrt(p_claim*(1-p_claim)/len(p_claim)), 0, 1)
upper = np.clip(p_claim + z*np.sqrt(p_claim*(1-p_claim)/len(p_claim)), 0, 1)

# Entropy = uncertainty
eps = 1e-12
entropy = -(p_claim*np.log(p_claim+eps) + (1-p_claim)*np.log(1-p_claim+eps))

print(f"{'Index':<8}{'P(claim)':<12}{'Lower95':<12}{'Upper95':<12}{'Entropy':<12}{'Actual':<8}")
print("-"*70)

for i in range(10):
    print(f"{i:<8}{p_claim[i]:<12.4f}{lower[i]:<12.4f}{upper[i]:<12.4f}{entropy[i]:<12.4f}{int(y_test.values[i]):<8}")



NGBOOST UNCERTAINTY (Classification)
Index   P(claim)    Lower95     Upper95     Entropy     Actual  
----------------------------------------------------------------------
0       0.3078      0.2173      0.3983      0.6173      0       
1       0.3856      0.2902      0.4810      0.6667      1       
2       0.1171      0.0541      0.1801      0.3611      0       
3       0.1992      0.1209      0.2775      0.4993      0       
4       0.2451      0.1607      0.3294      0.5568      1       
5       0.1606      0.0887      0.2326      0.4407      0       
6       0.2142      0.1338      0.2947      0.5195      0       
7       0.4337      0.3366      0.5308      0.6843      1       
8       0.2783      0.1904      0.3661      0.5913      1       
9       0.3246      0.2328      0.4163      0.6303      0       


# 03. EBM

In [ ]:
!pip install interpret

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [ ]:
print("\nTraining EBM...")
ebm_model = ExplainableBoostingClassifier(
    random_state=42,
    max_bins=256,
    interactions=10,
    n_jobs=-1,
    outer_bags=8,
    inner_bags=0
)

ebm_model.fit(X_train_encoded.values, y_train.values)

# Predict on validation
val_proba_ebm = ebm_model.predict_proba(X_val_encoded.values)[:, 1]

# Evaluate on validation
results_ebm_val = evaluate_model_comprehensive(
    y_val, val_proba_ebm, model_name="EBM (Validation)"
)


Training EBM...

EBM (Validation) - EVALUATION RESULTS
ROC-AUC:           0.6511
PR-AUC:            0.3784
Optimal Threshold: 0.210
F1-Score:          0.4452
Precision:         0.3174
Recall:            0.7450

Confusion Matrix:
[[3143 3606]
 [ 574 1677]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8456    0.4657    0.6006      6749
           1     0.3174    0.7450    0.4452      2251

    accuracy                         0.5356      9000
   macro avg     0.5815    0.6054    0.5229      9000
weighted avg     0.7135    0.5356    0.5617      9000



In [75]:
# Predict on test
test_proba_ebm = ebm_model.predict_proba(X_test_encoded.values)[:, 1]

# Evaluate on test
results_ebm_test = evaluate_model_comprehensive(
    y_test, test_proba_ebm,
    threshold=results_ebm_val['threshold'],
    model_name="EBM (Test)"
)


EBM (Test) - EVALUATION RESULTS
ROC-AUC:           0.6490
PR-AUC:            0.3666
Optimal Threshold: 0.210
F1-Score:          0.4391
Precision:         0.3124
Recall:            0.7387

Confusion Matrix:
[[3092 3658]
 [ 588 1662]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8402    0.4581    0.5929      6750
           1     0.3124    0.7387    0.4391      2250

    accuracy                         0.5282      9000
   macro avg     0.5763    0.5984    0.5160      9000
weighted avg     0.7083    0.5282    0.5545      9000



# EBM EXPLAINABILITY ANALYSIS

In [74]:
# Global explanation
ebm_global = ebm_model.explain_global()

# Extract feature importances
feature_importance = []
for i, feature_name in enumerate(RISK_FEATURES):
    importance = ebm_global.data(i)['scores']
    avg_importance = np.abs(importance).mean()
    feature_importance.append({
        'feature': feature_name,
        'importance': avg_importance
    })

importance_df = pd.DataFrame(feature_importance).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(importance_df.head(10).to_string(index=False))


Top 10 Most Important Features:
               feature  importance
accidents_last_3_years    0.390146
     doc_missing_score    0.145149
   risk_exposure_proxy    0.126839
     vehicle_age_years    0.118826
 registration_district    0.106429
        ncb_percentage    0.100162
    vehicle_usage_type    0.098505
       driver_age_band    0.091918
            driver_age    0.077677
          vehicle_type    0.068174


In [76]:
# Local explanation for a few samples
print("\n" + "="*80)
print("EBM EXPLAINABILITY - LOCAL EXPLANATIONS (Sample Predictions)")
print("="*80)

ebm_local = ebm_model.explain_local(X_test_encoded.values[:5], y_test.values[:5])

for i in range(5):
    local_data = ebm_local.data(i)
    print(f"\nSample {i+1}:")
    print(f"  Actual: {y_test.values[i]}")
    print(f"  Predicted Probability: {test_proba_ebm[i]:.4f}")
    print(f"  Top 3 Contributing Features:")

    # Get feature contributions
    contributions = []
    for j, feature_name in enumerate(local_data['names']):
        if feature_name != 'Intercept':
            contributions.append({
                'feature': feature_name,
                'value': local_data['values'][j],
                'score': local_data['scores'][j]
            })

    contrib_df = pd.DataFrame(contributions)
    contrib_df['abs_score'] = contrib_df['score'].abs()
    contrib_df = contrib_df.sort_values('abs_score', ascending=False)

    for _, row in contrib_df.head(3).iterrows():
        print(f"    {row['feature']}: {row['value']} (contribution: {row['score']:+.4f})")


EBM EXPLAINABILITY - LOCAL EXPLANATIONS (Sample Predictions)

Sample 1:
  Actual: 0
  Predicted Probability: 0.3554
  Top 3 Contributing Features:
    feature_0007: 1.0 (contribution: +0.1952)
    feature_0013: 20.0 (contribution: +0.1760)
    feature_0020: 1.0 (contribution: +0.1518)

Sample 2:
  Actual: 1
  Predicted Probability: 0.4394
  Top 3 Contributing Features:
    feature_0001: 0.0 (contribution: +0.2946)
    feature_0007: 0.0 (contribution: -0.2573)
    feature_0016: 0.0 (contribution: +0.1634)

Sample 3:
  Actual: 0
  Predicted Probability: 0.1069
  Top 3 Contributing Features:
    feature_0007: 0.0 (contribution: -0.2573)
    feature_0018: 0.0 (contribution: -0.2218)
    feature_0020: 1.0 (contribution: +0.1518)

Sample 4:
  Actual: 0
  Predicted Probability: 0.1884
  Top 3 Contributing Features:
    feature_0007: 0.0 (contribution: -0.2573)
    feature_0013: 24.0 (contribution: +0.2129)
    feature_0008: 50.0 (contribution: -0.1632)

Sample 5:
  Actual: 1
  Predicted Prob

**MODEL COMPARISON**

In [ ]:
comparison_results = pd.DataFrame([
    results_ft_test,
    results_ngb_test,
    results_ebm_test
])

print("\nTest Set Performance:")
print(comparison_results.to_string(index=False))

# Identify best model
best_model_idx = comparison_results['roc_auc'].idxmax()
best_model_name = comparison_results.loc[best_model_idx, 'model']

print(f"\n🏆 Best Model by ROC-AUC: {best_model_name}")
print(f"   ROC-AUC: {comparison_results.loc[best_model_idx, 'roc_auc']:.4f}")
print(f"   F1-Score: {comparison_results.loc[best_model_idx, 'f1']:.4f}")
print(f"   Recall: {comparison_results.loc[best_model_idx, 'recall']:.4f}")


Test Set Performance:
                model  roc_auc   pr_auc  threshold       f1  precision   recall
FT-Transformer (Test) 0.354010 0.186312       0.05 0.400000   0.250000 1.000000
       NGBoost (Test) 0.646390 0.366145       0.21 0.438549   0.308122 0.760444
           EBM (Test) 0.648995 0.366609       0.21 0.439102   0.312406 0.738667

🏆 Best Model by ROC-AUC: EBM (Test)
   ROC-AUC: 0.6490
   F1-Score: 0.4391
   Recall: 0.7387


RISK SCORE GENERATION

In [ ]:
# Use the best model's predictions
if 'FT-Transformer' in best_model_name:
    best_proba = test_proba_ft
elif 'NGBoost' in best_model_name:
    best_proba = test_proba_ngb
else:
    best_proba = test_proba_ebm

# Generate risk scores
risk_scores = (best_proba * 100).round().astype(int)

# Create results dataframe
results_df = pd.DataFrame({
    'actual_claim': y_test.values,
    'risk_probability': best_proba,
    'risk_score': risk_scores
})

# Categorize risk
def categorize_risk(score):
    if score < 20:
        return "Very Low"
    elif score < 40:
        return "Low"
    elif score < 60:
        return "Medium"
    elif score < 80:
        return "High"
    else:
        return "Very High"

results_df['risk_category'] = results_df['risk_score'].apply(categorize_risk)

print("\nSample Risk Scores:")
print(results_df.head(20).to_string(index=False))

print("\nRisk Score Distribution:")
print(results_df['risk_score'].describe())

print("\nRisk Category Distribution:")
print(results_df['risk_category'].value_counts())

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("\nKey Takeaways:")
print("1. FT-Transformer: Best for capturing complex feature interactions")
print("2. NGBoost: Provides uncertainty quantification with prediction intervals")
print("3. EBM: Fully interpretable model suitable for regulatory compliance")
print("\nAll three models complement your existing ensemble!")


Sample Risk Scores:
 actual_claim  risk_probability  risk_score risk_category
            0          0.355422          36           Low
            1          0.439378          44        Medium
            0          0.106891          11      Very Low
            0          0.188360          19      Very Low
            1          0.264247          26           Low
            0          0.142114          14      Very Low
            0          0.205784          21           Low
            1          0.415292          42        Medium
            1          0.279508          28           Low
            0          0.351792          35           Low
            0          0.162720          16      Very Low
            0          0.334559          33           Low
            0          0.500844          50        Medium
            0          0.179938          18      Very Low
            0          0.193883          19      Very Low
            0          0.124745          12      Ve